# Install dependencies

```
python3 -m venv env
source env/bin/activate
```

```
sudo apt-get install protobuf-compiler
```


```
PYTHONPATH=$PYTHONPATH:$(pwd)/models/TensorFlow/models/research/object_detection
echo "export PYTHONPATH=$PYTHONPATH" >> ~/.zshrc
```

In [1]:
%%bash
#pip3 install --upgrade --user tensorflow-gpu 
pip3 install --upgrade --user numpy
#pip3 install  --upgrade  --user tensorflow=="1.*"
#pip3 install --upgrade --pre --user tensorflow=="2.*"
#pip3 install --upgrade  --user pillow lxml matplotlib opencv-python pycocotools
#pip3 uninstall tensorflow=="2.*" -y

Requirement already up-to-date: numpy in /home/christian/.local/lib/python3.7/site-packages (1.17.4)


In [ ]:
!pip3 list | grep tensor

In [ ]:
%%bash
git clone https://github.com/tensorflow/models.git

In [ ]:
%%bash
cd models/research
python3 setup.py build 
python3 setup.py install 

# OpenCV
`OpenCV is a library of programming functions mainly aimed at real-time computer vision`

## Loop a local video

In [1]:
import cv2

cap = cv2.VideoCapture('../videos/test.mp4') 

if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if ret:
        # Display the resulting frame
        cv2.imshow('Frame',frame)

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        print('Loop restart')
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [17]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

# Tutorial
https://github.com/TannerGilbert/Tensorflow-Object-Detection-with-Tensorflow-2.0/blob/master/object_detection_with_webcam.ipynb

In [2]:
import sys
sys.path.append('../models/research/object_detection')

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import os
import pathlib

/home/christian/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/christian/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/christian/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/christian/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [3]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
        fname=model_name, 
        origin=base_url + model_file,
        untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [4]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '../models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

2.0.0-beta1


In [6]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
import cv2

# Define the video stream
cap = cv2.VideoCapture('../videos/test.mp4')  # Change only if you have more than one webcam
#cap = cv2.VideoCapture('./squirrel.mp4')

def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]
    
    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                   for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                                    output_dict['detection_masks'], output_dict['detection_boxes'],
                                    image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict



def run_inference(model, cap):
    print('Run int')
    while(cap.isOpened()):
        print('cap')
        ret, image_np = cap.read()
        # Actual detection.
        output_dict = run_inference_for_single_image(model, image_np)
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
            use_normalized_coordinates=True,
            line_thickness=8)
        cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
        # print the most likely
        max_label = category_index[1]
        max_score = output_dict['detection_scores'][0]#['name']
        print(max_label, max_score)
        
        # Quit on pressing q
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break



In [8]:
run_inference(detection_model, cap)

Run int
cap
{'id': 1, 'name': 'person'} 0.6139602
cap
{'id': 1, 'name': 'person'} 0.5894828
cap
{'id': 1, 'name': 'person'} 0.6196486
cap
{'id': 1, 'name': 'person'} 0.6259804
cap
{'id': 1, 'name': 'person'} 0.6162668
cap
{'id': 1, 'name': 'person'} 0.6529333
cap
{'id': 1, 'name': 'person'} 0.7139804
cap
{'id': 1, 'name': 'person'} 0.7132896
cap
{'id': 1, 'name': 'person'} 0.597062
cap
{'id': 1, 'name': 'person'} 0.53291565
cap
{'id': 1, 'name': 'person'} 0.64992094
cap
{'id': 1, 'name': 'person'} 0.73019886
cap
{'id': 1, 'name': 'person'} 0.6495251
cap
{'id': 1, 'name': 'person'} 0.6469361
cap
{'id': 1, 'name': 'person'} 0.6113709
cap
{'id': 1, 'name': 'person'} 0.6367143
cap
{'id': 1, 'name': 'person'} 0.6229569
cap
{'id': 1, 'name': 'person'} 0.7171195
cap
{'id': 1, 'name': 'person'} 0.55262566
cap
{'id': 1, 'name': 'person'} 0.6214308
